<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/Mistral_Engine_TensorRT_LLM_Builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

💡

https://pypi.org/project/tensorrt-llm/

tensorrt_llm don't support 3.11,  support 3.10 3.12, so need upgrade python to 3.12 or 3.10

# use python3.10

In [1]:
!ls /usr/bin/python3.*

/usr/bin/python3.10  /usr/bin/python3.10-config  /usr/bin/python3.11  /usr/bin/python3.11-config


In [2]:
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 2


update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in auto mode


In [3]:
!python --version

Python 3.10.12


In [ ]:
!apt-get install -y build-essential python3-dev python3-pip

In [10]:
!pip install -q -U pip setuptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00


# run

In [ ]:
# Clone the nvidia TensorRT LLM repository

!git clone https://github.com/NVIDIA/TensorRT-LLM.git
%cd TensorRT-LLM/examples/llama

In [ ]:
!pip install -q tensorrt_llm --extra-index-url https://pypi.nvidia.com

In [12]:
# Install necessary python dependencies
!pip install -q huggingface_hub pynvml mpi4py
!pip install -q -r requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 141.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 113.0 MB/s eta 0:00:00


In [13]:
!trtllm-build --help

2025-03-12 15:01:47,100 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
[TensorRT-LLM] TensorRT-LLM version: 0.18.0.dev2025031100
usage: trtllm-build [-h] [--checkpoint_dir CHECKPOINT_DIR] [--model_config MODEL_CONFIG]
                    [--build_config BUILD_CONFIG] [--model_cls_file MODEL_CLS_FILE]
                    [--model_cls_name MODEL_CLS_NAME] [--output_dir OUTPUT_DIR]
                    [--max_batch_size MAX_BATCH_SIZE] [--max_input_len MAX_INPUT_LEN]
                    [--max_seq_len MAX_SEQ_LEN] [--max_beam_width MAX_BEAM_WIDTH]
                    [--max_num_tokens MAX_NUM_TOKENS] [--opt_num_tokens OPT_NUM_TOKENS]
                    [--max_encoder_input_len MAX_ENCODER_INPUT_LEN]
                    [--max_prompt_embedding_table_size MAX_PROMPT_EMBEDDING_TABLE_SIZE]
                    [--kv_cache_type KV_CACHE_TYPE] [--paged_kv_cache PAGED_KV_CACHE]
                    [--input_timing_cache INPUT_TIMING_CACHE]
                    [--output_timi

In [15]:
# Download the Mistral 7B Instruct v0.2 weights from hugging face

from huggingface_hub import snapshot_download


snapshot_download(
    "mistralai/Mistral-7B-Instruct-v0.2",
    local_dir="/content/hf_models/mistral-7b-instruct-v0.2",
    max_workers=4
)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

'/content/hf_models/mistral-7b-instruct-v0.2'

In [16]:
# Convert the raw model weights into tensorrt-llm checkpoint format

!python convert_checkpoint.py --model_dir /content/hf_models/mistral-7b-instruct-v0.2 \
                             --output_dir /content/trt_engines/1-gpu/ \
                             --dtype float16

2025-03-12 15:04:52,898 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
[TensorRT-LLM] TensorRT-LLM version: 0.18.0.dev2025031100
0.18.0.dev2025031100
[03/12/2025-15:04:52] [TRT-LLM] [W] Implicitly setting LLaMAConfig.has_partial_lora_mask = False
[03/12/2025-15:04:52] [TRT-LLM] [W] Implicitly setting LLaMAConfig.tie_word_embeddings = False
230it [01:02,  3.66it/s]
Total time of reading and converting: 63.273 s
Total time of saving checkpoint: 151.182 s
Total time of converting checkpoints: 00:03:35


In [17]:
!ls -lh  /content/trt_engines/1-gpu

total 14G
-rw-r--r-- 1 root root 2.0K Mar 12 15:08 config.json
-rw-r--r-- 1 root root  14G Mar 12 15:08 rank0.safetensors


In [18]:
!cat  /content/trt_engines/1-gpu/config.json

{
    "mlp_bias": false,
    "attn_bias": false,
    "rotary_base": 1000000.0,
    "rotary_scaling": null,
    "residual_mlp": false,
    "disable_weight_only_quant_plugin": false,
    "moe": {
        "num_experts": 0,
        "shared_expert_intermediate_size": 0,
        "top_k": 0,
        "normalization_mode": null,
        "sparse_mixer_epsilon": 0.01,
        "tp_mode": 0,
        "device_limited_n_group": 0,
        "device_limited_topk_group": 0,
        "device_limited_routed_scaling_factor": 1.0
    },
    "remove_duplicated_kv_heads": false,
    "fc_after_embed": false,
    "use_input_layernorm_in_first_layer": true,
    "use_last_layernorm": true,
    "layer_idx_offset": 0,
    "embedding_multiplier": 1.0,
    "attention_multiplier": 1.0,
    "residual_multiplier": 1.0,
    "output_multiplier_scale": 1.0,
    "has_partial_lora_mask": false,
    "architecture": "MistralForCausalLM",
    "dtype": "float16",
    "vocab_size": 32000,
    "hidden_size": 4096,
    "num_hidden_lay

In [19]:
# Compile the model

!trtllm-build --checkpoint_dir /content/trt_engines/1-gpu/ \
            --output_dir /content/trt_engines/compiled-model/ \
            --gpt_attention_plugin float16 \
            --gemm_plugin float16 \
            --max_input_len 32256


2025-03-12 15:12:07,207 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
[TensorRT-LLM] TensorRT-LLM version: 0.18.0.dev2025031100
[03/12/2025-15:12:07] [TRT-LLM] [I] Set bert_attention_plugin to auto.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set gpt_attention_plugin to float16.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set gemm_plugin to float16.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set gemm_swiglu_plugin to None.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set fp8_rowwise_gemm_plugin to None.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set nccl_plugin to auto.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set lora_plugin to None.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set dora_plugin to False.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set moe_plugin to auto.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set mamba_conv1d_plugin to auto.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set low_latency_gemm_plugin to None.
[03/12/2025-15:12:07] [TRT-LLM] [I] Set low_latency_gemm_swiglu_plugin to None.
[03/12/2025-15:12:07] [TRT-LLM] 

In [20]:
!ls -hl /content/trt_engines/compiled-model/

total 14G
-rw-r--r-- 1 root root 6.3K Mar 12 15:13 config.json
-rw-r--r-- 1 root root  14G Mar 12 15:18 rank0.engine


In [21]:
!cat /content/trt_engines/compiled-model/config.json

{
    "version": "0.18.0.dev2025031100",
    "pretrained_config": {
        "mlp_bias": false,
        "attn_bias": false,
        "rotary_base": 1000000.0,
        "rotary_scaling": null,
        "residual_mlp": false,
        "disable_weight_only_quant_plugin": false,
        "moe": {
            "num_experts": 0,
            "shared_expert_intermediate_size": 0,
            "top_k": 0,
            "normalization_mode": null,
            "sparse_mixer_epsilon": 0.01,
            "tp_mode": 0,
            "device_limited_n_group": 0,
            "device_limited_topk_group": 0,
            "device_limited_routed_scaling_factor": 1.0
        },
        "remove_duplicated_kv_heads": false,
        "fc_after_embed": false,
        "use_input_layernorm_in_first_layer": true,
        "use_last_layernorm": true,
        "layer_idx_offset": 0,
        "embedding_multiplier": 1.0,
        "attention_multiplier": 1.0,
        "residual_multiplier": 1.0,
        "output_multiplier_scale": 1.0,
 

In [25]:
# Upload the compiled model to hugging face hub

import os
from huggingface_hub import HfApi, create_repo
from google.colab import userdata

# Get your Hugging Face token from userdata
hf_token = userdata.get('HF_TOKEN')

# Create the repository if it doesn't exist
create_repo(repo_id="weege007/mistral-7b-instruct-v0.2-trtllm", token=hf_token, exist_ok=True)

# Initialize the HfApi with your token
api = HfApi(token=hf_token)

for root, dirs, files in os.walk(f"/content/trt_engines/compiled-model", topdown=False):
    for name in files:
        filepath = os.path.join(root, name)
        filename = "/".join(filepath.split("/")[-2:])
        print("uploading file: ", filename)

        # Upload the file
        api.upload_file(
            path_or_fileobj=filepath,
            path_in_repo=filename,
            repo_id="weege007/mistral-7b-instruct-v0.2-trtllm",
            create_pr=True,
            repo_type="model",
        )

uploading file:  compiled-model/rank0.engine


rank0.engine:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

uploading file:  compiled-model/config.json


In [26]:
%cd ..

/content/TensorRT-LLM/examples


In [27]:
# Test the compiled model

!python3 run.py --max_output_len=256 \
               --tokenizer_dir /content/hf_models/mistral-7b-instruct-v0.2/ \
               --engine_dir=/content/trt_engines/compiled-model \
               --max_attention_window_size=4096

2025-03-12 15:39:54,965 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
[TensorRT-LLM] TensorRT-LLM version: 0.18.0.dev2025031100
[TensorRT-LLM][INFO] Engine version 0.18.0.dev2025031100 found in the config file, assuming engine(s) built by new builder API.
[03/12/2025-15:39:55] [TRT-LLM] [I] Using C++ session
[TensorRT-LLM][INFO] Engine version 0.18.0.dev2025031100 found in the config file, assuming engine(s) built by new builder API.
[TensorRT-LLM][INFO] MPI size: 1, MPI local size: 1, rank: 0
[03/12/2025-15:39:55] [TRT-LLM] [W] Implicitly setting LLaMAConfig.fc_after_embed = False
[03/12/2025-15:39:55] [TRT-LLM] [W] Implicitly setting LLaMAConfig.use_input_layernorm_in_first_layer = True
[03/12/2025-15:39:55] [TRT-LLM] [W] Implicitly setting LLaMAConfig.use_last_layernorm = True
[03/12/2025-15:39:55] [TRT-LLM] [W] Implicitly setting LLaMAConfig.layer_idx_offset = 0
[03/12/2025-15:39:55] [TRT-LLM] [W] Implicitly setting LLaMAConfig.has_partial_lora_mask = False